# LAV-ESG fileserver access: getting started

This is a jupyter notebook intended for all students working with the fileserver data of the Energy Systems Group of LAV. In this short document you should get a first impression of what a notebook file is, how it looks like and you should learn how to connect to the fileserver to access data.

## About jupyter notebook

It is an environment consisting of a **cell based** structure allowing to combine documentation (Markdown) and coding. Each cell can be accessed by *double click* and executed by pressing *Shift + Enter* or *Ctrl + Enter*.

There are many cool things possible, like:
- bulleting
- $latex$
- and its math tool: $ e^{\pi \cdot i} + 1 = 0$
- even centered $$ e^{\pi \cdot i} + 1 = 0$$
- references and links to files or webpages. You find the official [description here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/index.html). Just check it out.

There are many [tutorials](https://nbviewer.jupyter.org/github/ipython/ipython/blob/4.0.x/examples/IPython%20Kernel/Index.ipynb) online - either directly from *Jupyter*, provided by github or tutorial videos. Just google for them. After you went through this notebook it is recommended to start with a tutorial. 

Of course you can execute some python code directly in the notebook (or load it from an external source). Make yourself familiar with shortcuts to quickly switch between *Code* and *Markdown* cells. You can see the list of shortcuts when you press *h*. Knowing this, we finally want to come up with some coding! First, let us import a package and rename it (you need to execute the cells below, having *IN [ ]* in front of them):

In [ ]:
import numpy as np

This package allows us for instance to perform array multiplication (element-wise):

In [ ]:
# define the arrays
a = np.array([2, 4, 6])
b = np.array([1, 2, 1])

In [ ]:
c = a*b

We can see the result by printing it (or giving it directly)

In [ ]:
c

In [ ]:
print(c)

And it acutally works. This should just be a trivial example to let you see how those cells work AND that packages can be imported. Such a package is the one you just installed in STEP 3 (point 2) of the *LAV-ESG_gettingStarted* document. This package provides useful commands to access the fileserver and its data. Let's see how it works...

## Access the LAV-fileserver

First (again), import the needed package (LAV extension) by executing the cell below:

In [ ]:
%load_ext ipython_pg.ipython_extension_LAV

and connect to the server, namely the *mobility* database (**HERE** you are asked to enter your password. You need your separate password from your supervisor. It is important not to have access passwords stored anywhere in a code file or notebook, but use this line magic to connect to the database):

In [ ]:
%pg_connect host='lav-fileserver' port='5433' dbname='mobility' sslcert=sslcert

If you installed everything correctly, there should be the line:

    SUCCESS: connected to lav-fileserver

Now we can access the data with *queries*, which return *cursors* pointing to the data. Below you will find some examples how it can be done. Run them and try to understand how they work. If you need help with the *SQL* language, search for tutorials or documentation online.

### cell magic with cursor

The cell magic (*%%pg_sql*) allows you to write your query directly in one cell and execute it. In the example below, we are accessing the schema *mzmv2010* (Mikrozensus Mobilität und Verkehr 2010) and within it the table *Wege*. The data we are *selecting* is the *ausnr* (OID/number of trips of one person at the survey day) and *count* how often each number appears in the table. To do that, we need to *group* the data for each *ausnr*. The word after the cell magic is the name of the cursor which will be returned, here: *cur*

In [ ]:
%%pg_sql cur
SELECT w."ausnr", count(w."ausnr") FROM mzmv2010.Wege w GROUP BY w."ausnr" ORDER BY w."ausnr"

If everything works properly you should see:

    SUCESS: matched 12 rows
     cursor object as 'cur'
   
This means we got a cursor returned consisting of 12 rows. We can now extract the data by iterating the cursor, and e.g. store it as tuples in a list:

In [ ]:
trip_Numbers = []
for row in cur:
    trip_Numbers.append((row[0], row[1]))

If you want to see how the *trip_Numbers* list looks like, we can print its content

In [ ]:
for item in trip_Numbers:
    print("ausnr {: 3.0f} is contained {: 7d} times".format(item[0], item[1]))

This should just be an example how you access the data and then further process it within your python code. Your query will be most likely more complexe.

### cell magic without cursor

In the example above, we basically just wanted to see the result of the query and are not using the data any further. With the LAV extension this can be done directly, if you don't return a cursor. Thus leave away the word after the cell magic and the results of the query will get illustrated

In [ ]:
%%pg_sql
SELECT w."ausnr", count(w."ausnr") FROM mzmv2010.Wege w GROUP BY w."ausnr" ORDER BY w."ausnr"

See, we got the exact same outcome directly. This is useful when you're just interessted in investigating the data. You can do that with any kind of query. E.g. below we spice it up a bit, where we look into the trips having the number *-98* (which are incomplete trips). We start from the *Etappen* table and only consider valid stages performed with a car in Switzerland when the car is a business car (the 4 conditions after *WHERE*). If such a stage belongs to a trip with number *-98*, we'd like to see all the stages of the corresponding person (as an array --> *array_agg*) and to which trip number each stage belongs. We don't have the *ausnr* in the *Etappen* table though, this is why we need to join it with the *Wege* table on matching (defining) variables (*HHNR, ZPNR, WEGNR*) and get it from there. Just run the cell and see what happens

In [ ]:
%%pg_sql
SELECT e."HHNR", e."ZIELPNR", array_agg(w."ausnr"  ORDER BY e."f51100") as Ausgaenge -- # identify the vehicle
FROM mzmv2010.Etappen e
LEFT JOIN mzmv2010.Wege w ON w."WEGNR"=e."WEGNR" AND w."ZIELPNR"=e."ZIELPNR" AND w."HHNR"=e."HHNR"
WHERE e."pseudo"=1   -- gültige etappe
AND e."f51300"=7     -- benutztes Verkehrsmittel: Auto als Fahrer'
AND e."E_Ausland"=2  -- Auslandsetappe: Nein
AND e."F51310a"=2    -- Firmenauto
GROUP BY e."HHNR", e."ZIELPNR"
HAVING min(w."ausnr")=-98

Again, it's just an example. Nevertheless, it should help you to get the picture how you can use this extension. Of course you don't have to use the cell magic, but can execute queries based on strings. An example of this method is shown below.

### sql string

We can run the same query from above, listing the query as a sequence of strings:

In [ ]:
sql = ('SELECT w."ausnr", count(w."ausnr")',
       'FROM mzmv2010.Wege w',
       'GROUP BY w."ausnr"',
       'ORDER BY w."ausnr"'
      )

try:
    cur = %pg_cursor
    cur.execute(" ".join(sql))
except Exception as e:
    print(str(e))
    %pg_rollback()

And print results just as above:

In [ ]:
for item in cur:
    print("ausnr {: 3.0f} is contained {: 7d} times".format(item[0], item[1]))

how you segment (or if) is up to you. It can bring some advantages with it though. The advantage of using the string form to run queries is the easy modification of the sql to run similar queries in one cell, like the one below. Here we simply want to count how many stages are driven with a car of the houshold, with a business car and so on. The only condition that changes is the type of vehicle used. Therefore we can modify the sql string in a for loop and execute the query multiple times.

In [ ]:
sql = ('SELECT COUNT(e."WP")',
       'FROM mzmv2010.Etappen e',
       'WHERE e."pseudo"=1',  # gültige Etappe
       'AND e."f51300"=7',    # benutztes Verkehrsmittel: Auto als Fahrer'
       'AND e."E_Ausland"=2', # Auslandsetappe: Nein
       'AND e."F51310a"'      # offene Bedingung (= *Zahl* fehlt)
      )
sql = " ".join(sql)

# F51310a: benutztes Auto (1=im Haushalt, 2=Firmenauto, 3=Mieatuto, 4=Carsharing, 5=andere, <0=N/A)
for text, cond in (("Im Haushalt", "=1"), ("Firmenauto", "=2"), ("Geliehen", ">2"), 
                   ("Keine Angabe", "IN (-97, -98)"), ("Fehler", "=-99"),
                   (None, None), ("Total", "< 100")):
    
    if text is None:
        print("-"*22)
        continue
        
    try:
        cur = %pg_cursor
        cur.execute(" ".join((sql, cond)))
        print("{:15s} {: 6d}".format("{}:".format(text), *cur.fetchone()))
    except Exception as e:
        print(str(e))
        %pg_rollback()

This is the end of the gettingStarted notebook. We hope we were able to show you the functioning principle and usability of the jupyter notebook in combination with the LAV extension to access the LAV fileserver. It is up to you what environment you are using, as long as you are able to run sql queries. Good luck and have fun with your project!